In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

In [2]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import sys
import logging
import ipytest
from pathlib import Path

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import logging

from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

# Update this variable to your bucket name. This name must be unique
# across all AWS accounts.
BUCKET = os.environ["BUCKET"]
S3_LOCATION = f"s3://{BUCKET}/football"

# To run this notebook in Local Model, this constant must be set to True.
# I'm trying to do this automatically by checking for a specific environment
# variable that is set by SageMaker when you run the notebook inside SageMaker
# Studio. 
# LOCAL_MODE = "SAGEMAKER_INTERNAL_IMAGE_URI" not in os.environ

LOCAL_MODE = False

# This variable will be used to determine the architecture of the
# local machine. If the machine is an ARM64 machine, you will need
# to build a custom Docker image using the setup notebook.
ARCHITECTURE = !(uname -m)

# This is a dummy role that will be ignored when we run the
# pipeline in Local Mode.
DUMMY_ROLE = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-11111111111111"

# We'll use these two variables to configure the steps that do not support
# Local Mode.
pipeline_session = PipelineSession() if not LOCAL_MODE else LocalPipelineSession(default_bucket=BUCKET)
execution_role = os.environ["ROLE"] if not LOCAL_MODE else DUMMY_ROLE

if LOCAL_MODE:
    config = {
        "session": pipeline_session,
        "instance_type": "local",
        "role": DUMMY_ROLE,

        # We need to use a custom Docker image when we run the pipeline
        # in Local Model on an ARM64 machine.
        "image": "sagemaker-tensorflow-training-toolkit-local" if ARCHITECTURE[0] == "arm64" else None,
        "framework_version": None if ARCHITECTURE[0] == "arm64" else "1.7-1",
        "py_version": None if ARCHITECTURE[0] == "arm64" else "py38",
    }
else:
    config = {
        "session": pipeline_session,
        "instance_type": "ml.c5.xlarge",
        "role": execution_role,
        "image": None,
        "framework_version": "1.7-1",
        "py_version": "py3",
    }

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

In [5]:
import boto3

s3_client = boto3.client('s3')
iam_client = boto3.client("iam")
sagemaker_client = boto3.client("sagemaker")

In [6]:
from sagemaker.s3 import S3Uploader

df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])
S3Uploader.upload(local_path=y_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])

's3://football-data-kamil/football/data/y.csv'

In [7]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

In [8]:
from sagemaker.sklearn import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.retry import (
    SageMakerJobExceptionTypeEnum,
    SageMakerJobStepRetryPolicy
)

retry_policy = {
    "ExceptionType": SageMakerJobExceptionTypeEnum.INTERNAL_ERROR,
    "IntervalSeconds": 2,
    "BackoffRate": 2,
    "MaxAttempts": 5,
    "ExpireAfterMin": 5
}

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

processor = SKLearnProcessor(
    base_job_name="split-and-transform-data",
    framework_version="1.2-1",
    instance_type=config["instance_type"],
    instance_count=1,
    role=config["role"],
    sagemaker_session=config["session"],
)

split_and_transform_data_step = ProcessingStep(
    name="split-and-transform-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),
        ],
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
            ProcessingOutput(output_name="model", source="/opt/ml/processing/model"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline"),
        ]
    ),
    cache_config=cache_config,
)



In [9]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

# session1_pipeline = Pipeline(
#     name="session1-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         split_and_transform_data_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config['session'],
# )
# 
# session1_pipeline.upsert(role_arn=config["role"])
# session1_pipeline.start()

In [10]:
from sagemaker.xgboost import XGBoost

use_spot_instances = True and not LOCAL_MODE
max_run = 1000
max_wait = 1200 if use_spot_instances else None
instance_type = "ml.m5.2xlarge" 

estimator = XGBoost(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    role=config['role'],
    instance_count=1,
    instance_type=config['instance_type'],
    framework_version=config['framework_version'],
    disable_profiler=True,
    py_version=config['py_version'],
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    metric_definitions=[
        {'Name': 'validation:logloss', 'Regex': '.*\[[0-9]+\].*#011validation_0-logloss:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
        {'Name': 'validation:f1', 'Regex': 'F1 score: ([0-9\\.]+)'}
    ],
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [22]:
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput

# train_model_step = TrainingStep(
#     name="train-model",
#     estimator=estimator,
#     inputs={
#         "train": TrainingInput(
#             s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
#             content_type="text/csv"
#         ),
#         "validation": TrainingInput(
#             s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
#             content_type="text/csv"
#         )
#     },
#     cache_config=cache_config
# )

uri = split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri.to_string()

s_uri = split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri.to_string()

s_uri
# train_model_step = TrainingStep(
#     name="train-model",
#     step_args=estimator.fit(
#         inputs={
#             "train": TrainingInput(
#                 s3_data=s_uri,
#                 content_type="text/csv"
#             ),
#             "validation": TrainingInput(
#                 s3_data=uri,
#                 content_type="text/csv"
#             )
#         }
#     ),
#     cache_config=cache_config
# )

Join(on='', values=[{'_step': <sagemaker.workflow.steps.ProcessingStep object at 0x0000013F50F07230>, 'step_name': 'split-and-transform-data', 'path': "ProcessingOutputConfig.Outputs['train'].S3Output.S3Uri", '_shape_names': ['S3Uri'], '__str__': 'S3Uri'}])

In [11]:
USE_TUNING_STEP = False and not LOCAL_MODE

In [12]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.parameter import IntegerParameter, ContinuousParameter

hyperparameter_ranges = {
    'eta': ContinuousParameter(min_value=0.05, max_value=0.3, scaling_type="Logarithmic"),
    'max_depth': IntegerParameter(min_value=5, max_value=15, scaling_type="Auto"),
    'subsample': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Auto"),
    'colsample_bytree': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Logarithmic"),
    'lambda': ContinuousParameter(min_value=5, max_value=12, scaling_type="Logarithmic"),
    'alpha': ContinuousParameter(min_value=1, max_value=10, scaling_type="Logarithmic"),
    'min_child_weight': ContinuousParameter(min_value=0.4, max_value=1.0, scaling_type="Auto"),
}

objective_type = "Maximize"
metric_definitions = [
    {'Name': 'validation:logloss',
     'Regex': '.*\[[0-9]+\].*#011validation_0-logloss:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'
     },
    {'Name': 'validation:f1',
     'Regex': 'F1 score: ([0-9\\.]+)'
     },
]
metric_name = "validation:f1"
strategy = "Bayesian"

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=8,
    max_parallel_jobs=2,
    early_stopping_type='Auto',
)

NameError: name 'estimator' is not defined

In [ ]:
from sagemaker.workflow.steps import TuningStep

# tune_model_step = TuningStep(
#     name="tune-model",
#     tuner=tuner,
#     inputs={
#         "train": TrainingInput(
#             s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
#             content_type="text/csv"
#         ),
#         "validation": TrainingInput(
#             s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
#             content_type="text/csv"
#         )
#     },
#     cache_config=cache_config
# )

tune_model_step = TuningStep(
    name="tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        },
    ),
    cache_config=cache_config
)


In [ ]:
session2_pipeline = Pipeline(
    name="session2-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session2_pipeline.upsert(role_arn=config['role'])
session2_pipeline.start()

In [16]:
from sagemaker.xgboost import XGBoostProcessor

evaluation_processor = XGBoostProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    py_version=config["py_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=config['role'],
    sagemaker_session=config["session"],
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [17]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

In [18]:
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=evaluation_processor.run(
        inputs=[
            ProcessingInput(
                source=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            ProcessingInput(
                source=tune_model_step.get_top_model_s3_uri(
                    top_k=0, s3_bucket=config["session"].default_bucket()
                ) if USE_TUNING_STEP else train_model_step.properties.ModelArtifacts.S3ModelArtifacts,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

In [19]:
# Code to retrieve last model metric.
# from sagemaker.workflow.parameters import ParameterFloat
# import json
# 
# response = sagemaker_client.list_model_packages(ModelPackageGroupName=MODEL_PACKAGE_GROUP,
#                                                 SortBy='CreationTime',
#                                                 SortOrder='Descending',
#                                                 MaxResults=1)
# 
# if response['ModelPackageSummaryList']:
#     latest_model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
#     model_package_description = sagemaker_client.describe_model_package(ModelPackageName=latest_model_package_arn)
#     evaluation_metrics = model_package_description['ModelMetrics']['ModelQuality']['Statistics']
# 
#     s3_uri = evaluation_metrics['S3Uri']
#     s3_bucket, s3_key = s3_uri.replace("s3://", "").split("/", 1)
# 
#     obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
#     evaluation_json = json.loads(obj['Body'].read())
#     f1_metric = evaluation_json['metrics']['f1']['value']
# 
#     f1_threshold = ParameterFloat(name="f1_threshold", default_value=f1_metric)
#     print(f"F1 score from the latest model's evaluation: {f1_metric}")
# else:
#     print("No models found in the specified model package group.")
#     f1_metric = 0.67
#     f1_threshold = ParameterFloat(name="f1_threshold", default_value=f1_metric)

In [20]:
from sagemaker.workflow.functions import Join

transformation_pipeline_model = Join(
    on="/",
    values=[
        split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "model"
        ].S3Output.S3Uri,
        "model.tar.gz",
    ],
)

In [21]:
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn.model import SKLearnModel

preprocessing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="preprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)
xgb_model = XGBoostModel(
    model_data=tune_model_step.get_top_model_s3_uri(
        top_k=0, s3_bucket=config["session"].default_bucket()
    ) if USE_TUNING_STEP else train_model_step.properties.ModelArtifacts.S3ModelArtifacts,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=config["role"],
)

post_processing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="postprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)

from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name="inference-model",
    models=[preprocessing_model, xgb_model, post_processing_model],
    sagemaker_session=config["session"],
    role=config['role'],
)

In [22]:
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import DataQualityCheckConfig, QualityCheckStep
from sagemaker.model_monitor.dataset_format import DatasetFormat

DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"

data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",

    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=config['session'],
        role=config['role'],
    ),

    quality_check_config=DataQualityCheckConfig(
        baseline_dataset=f"{S3_LOCATION}/data",
        dataset_format=DatasetFormat.csv(header=True, output_columns_position="END"),
        output_s3_uri=DATA_QUALITY_LOCATION
    ),

    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [23]:
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.transformer import Transformer

create_model_step = ModelStep(
    name="create-model-step",
    display_name="create-model",
    step_args=pipeline_model.create(
        instance_type="ml.m5.xlarge"
    ),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    instance_type=config["instance_type"],
    instance_count=1,
    strategy="MultiRecord",
    accept="text/csv",
    assemble_with="Line",
    output_path=f"{S3_LOCATION}/transform",
    sagemaker_session=config["session"]
)

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    step_args=transformer.transform(
        # We will use the baseline set we generated when we split the data.
        # This set corresponds to the test split before the transformation step.
        data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["baseline"].S3Output.S3Uri,

        join_source="Input",
        split_type="Line",
        content_type="text/csv",

        # We want to output the first and the last field from the joint set.
        # The first field corresponds to the groundtruth, and the last field
        # corresponds to the prediction.
        output_filter="$[0,-1]",
    ),
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.


In [24]:
from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

MODEL_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/model-quality"

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",

    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=pipeline_session,
        role=execution_role,
    ),

    quality_check_config=ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,
        dataset_format=DatasetFormat.csv(header=False),

        # We need to specify the problem type and the fields where the prediction
        # and groundtruth are so the process knows how to interpret the results.
        problem_type="BinaryClassification",

        # Since the data doesn't have headers, SageMaker will autocreate headers for it.
        # _c0 corresponds to the first column, and _c1 corresponds to the second column.
        ground_truth_attribute="_c0",
        inference_attribute="_c1",

        output_s3_uri=MODEL_QUALITY_LOCATION,
    ),

    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [25]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.drift_check_baselines import DriftCheckBaselines

model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),

    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

In [26]:
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline"

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=pipeline_model.register(
        model_package_group_name=PIPELINE_MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="PendingManualApproval",
        content_types=["text/csv", "application/json"],
        response_types=["text/csv", "application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="XGBOOST",
        framework_version=config["framework_version"],
    ),
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [27]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep

accuracy_threshold = ParameterFloat(name="f1_score_threshold", default_value=0.64)

condition_step = ConditionStep(
    name="check-model-f1_score",
    conditions=[
        ConditionGreaterThanOrEqualTo(
            left=JsonGet(
                step_name=evaluate_model_step.name,
                property_file=evaluation_report,
                json_path="metrics.f1.value",
            ),
            right=accuracy_threshold,
        )
    ],
    if_steps=[],

    else_steps=[
        FailStep(
            name="fail",
            error_message=Join(
                on=" ",
                values=[
                    "Execution failed because the model's accuracy was lower than",
                    accuracy_threshold,
                ],
            ),
        )
    ],
)

if not LOCAL_MODE:
    # SageMaker doesn't support running any of these steps in Local Mode.
    condition_step.if_steps.extend(
        [
            create_model_step,
            generate_test_predictions_step,
            model_quality_baseline_step,
            register_model_step,
        ]
    )

In [28]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline = Pipeline(
    name="session4-pipeline",
    parameters=[
        dataset_location,
        accuracy_threshold,
    ],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step
    ],
    pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True),
    sagemaker_session=config["session"],
)

if not LOCAL_MODE:
    # SageMaker doesn't support running any of these steps in Local Mode.
    pipeline.steps.extend([data_quality_baseline_step])

pipeline.upsert(role_arn=config["role"])

INFO:sagemaker.processing:Uploaded None to s3://sagemaker-eu-north-1-284415450706/session4-pipeline/code/a01b8389f6ae7461a33ca11537e1facf/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-north-1-284415450706/session4-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-eu-north-1-284415450706/session4-pipeline/code/a01b8389f6ae7461a33ca11537e1facf/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-eu-north-1-284415450706/session4-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session4-pipeline',
 'ResponseMetadata': {'RequestId': '6720208d-fc08-48fa-9fc4-ba4a506c95cd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6720208d-fc08-48fa-9fc4-ba4a506c95cd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Mon, 15 Apr 2024 11:58:45 GMT'},
  'RetryAttempts': 0}}

In [29]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session4-pipeline/execution/9jfdxhlkr9d9', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x000001EE9261F860>)